### Tensorflow

- Tensorflow GPU 는 Python 3.10 까지만 지원

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.18.0'

#### GPU 확인

In [2]:
import os
import cv2 
import tensorflow as tf 
import numpy as np
from tensorflow.python.client import device_lib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 324684948807901285
 xla_global_id: -1]

#### ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz 다운로드 
- https://github.com/wikibook/opencv4/blob/opencv4-rev/Chapter10/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz 에서 다운로드
- 압축해제 및 연결

In [6]:
model = tf.saved_model.load("../data/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model")

#### 샘플 mp4 다운로드
- https://github.com/wikibook/opencv4/blob/opencv4-rev/Chapter10/Example-10-03-Py/bird.mp4 에서 다운로드

##### 추론결과 표시

In [7]:
# model = tf.saved_model.load("../data/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model")
capture = cv2.VideoCapture('../data/bird_720p.mp4')

while True:
    _, frame = capture.read()

    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        break

    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    input_tensor = tf.convert_to_tensor(input_img)
    input_tensor = input_tensor[tf.newaxis, ...]

    output_dict = model.signatures["serving_default"](input_tensor)

    classes = output_dict["detection_classes"][0]
    scores = output_dict["detection_scores"][0]
    boxes = output_dict["detection_boxes"][0]

    height, width, _ = frame.shape
    for idx, score in enumerate(scores):
        if score > 0.7:
            class_id = int(classes[idx])
            box = boxes[idx]

            x1 = int(box[1] * width)
            y1 = int(box[0] * height)
            x2 = int(box[3] * width)
            y2 = int(box[2] * height)

            cv2.rectangle(frame, (x1, y1), (x2, y2), 255, 1)
            cv2.putText(frame, str(class_id) + ":" + str(float(score)), (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 255, 255), 1)

    cv2.imshow("Object Detection", frame)
    if cv2.waitKey(0) == ord('q'): break

InvalidArgumentError: Graph execution error:

Detected at node Preprocessor/ResizeImage/resize/ResizeBilinear defined at (most recent call last):
<stack traces unavailable>
input must be 4-dimensional[1,720,1280]
	 [[{{node Preprocessor/ResizeImage/resize/ResizeBilinear}}]] [Op:__inference_signature_wrapper_145518]